In [ ]:
#! /usr/bin/python
# -*- coding: utf-8 -*-

import time
import io
import cv2
import numpy
import re
import sqlite3
import dlib
import shutil
import zipfile
import glob
import os


from PIL import Image
from http.server import BaseHTTPRequestHandler, HTTPServer
from os import curdir
from os.path import join as pjoin
from scipy.spatial import distance
from random import choice
from string import ascii_letters
from string import digits



HOST_NAME = 'localhost' # !!!REMEMBER TO CHANGE THIS!!!
PORT_NUMBER = 1111 # Maybe set this to 9000.
DB_NAME = 'faces.db'

def searchInDb(desc):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    selectedData = []
    sel = cursor.execute("select id, desc from face")
    targetIndex = 0
    for line in sel:
        if len(desc) != 0:
            euclidianDistanse = distance.euclidean(numpy.array(line[1].split('||'), dtype=float), numpy.array(desc.split('||'), dtype=float))
            if euclidianDistanse < 0.6:
                targetIndex = line[0]
                print(euclidianDistanse)
            else:
                print(str(euclidianDistanse) + '> 0.6')
    
    result = ''
    if targetIndex!=0:
        sel = cursor.execute("select rawstring from face where id=?", [targetIndex])
        for line in sel:
            result = line[0]    
    conn.close()
    return result

def create_key():
    key = "".join(choice(ascii_letters + digits) for i in range(20))
    return key

def appendToDb(keywords, rawstring, desc, image):
    if desc != '':
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        sel = cursor.execute("SELECT key FROM face")
        sel_list = [i[0] for i in sel]
        key = create_key()
        if sel_list.count(key) > 1:
            while True:
                key = create_key()
                if sel_list.count(key) == 1:
                    break
        
        ins = cursor.execute("INSERT INTO face (rawstring, keywords, date, desc, key) VALUES( ?, ?, ?, ?, ?)",
                         (rawstring, keywords[0], keywords[1], desc, key))
        conn.commit()
        conn.close()
        image_name = 'images\\%s.png' % key 
        print(image_name)
        cv2.imwrite(image_name, image)

def getDescriptor(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    detector = dlib.get_frontal_face_detector()
    dets = detector(gray, 1)
    sp = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
    facerec = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')
    faceDescriptor = ''
    for i, d in enumerate(dets):
            if len(dets) != 2:
                shape = sp(img, d)
                faceDescriptor += str(facerec.compute_face_descriptor(img, shape)).replace('\n', '||')
                if i != len(dets) - 1:
                    faceDescriptor += '\n'
    return faceDescriptor

def parseDataFromPost(stringToParse):
    print(stringToParse)
    keywords = ""
    date = ""
    splitted = stringToParse.split('<<row>>')
    for line in splitted:
        if line != '':
            keywords += line.split('=')[1] + '<<key>>' 
            if line.split('=')[0] == "Дата регистрации":
                date = line.split('=')[1]
    print(date)
    return keywords, date

def countOfRowsInDB():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    count = 0
    res = cursor.execute("SELECT COUNT(*) FROM face")
    for line in res:
        count = line
    conn.close()
    return count[0]

def textSearchingInDB(searchString):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    results = []
    res = cursor.execute("SELECT id, keywords FROM face")
    full_select = [line[::1] for line in res]
    conn.close()
#     print(full_select)
    for line in full_select:
        if re.search(searchString.upper(), line[1].upper()):
            results.append(line[0])
    if len(results) == 0:
        return(0)
    else:
        return(results)

def prepare_searched_result(searchString):
    searched_results = textSearchingInDB(searchString)
    result = []
    images_names = []
    ids = []
    if searched_results != 0:
        for line in searched_results:
            conn = sqlite3.connect(DB_NAME)
            cursor = conn.cursor()
            sel = cursor.execute("select rawstring, key, id from face where id=?", [line])
            sel = [i for i in sel]
            result.append([i[0] for i in sel])
            images_names.append([i[1] for i in sel])
            ids.append([i[2] for i in sel])
            conn.close()
        return result, images_names, ids
    else:
        return result, images_names, ids

def prepare_searched_response(searchString):
    string, image_names, ids = prepare_searched_result(searchString)
    images_bytes = []
    file_names = []
    result = b''
    
    if len(string) != 0 and len(image_names) != 0 and len(ids) !=0:
        for i, line in enumerate(image_names):
            shutil.copy('images\\' + line[0] + '.png', 'archive_dir\\' + line[0] + '.png')
            filename = line[0] + '.png'
            file_names.append('archive_dir\\' + filename)
            result += str(ids[i][0]).encode("utf-8") + b'<id>' + string[i][0].encode("utf-8") + b'<filename>' + filename.encode("utf-8") + b'<end>'
        with zipfile.ZipFile('archive_dir\\file.zip', 'w') as myzip:
            for line in file_names:
                myzip.write(line)
        file = open('archive_dir\\file.zip', 'rb')
        file_bytes = file.read()
        file.close()
        print(len(file_bytes))
        files = glob.glob('archive_dir\\*')
        for f in files:
            os.remove(f)
        return file_bytes, result.decode('unicode-escape').encode('ISO-8859-1')
    else:
        return file_names, result.decode('unicode-escape').encode('ISO-8859-1')

def delete_record(request_string):
    print(request_string)
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    sel = cursor.execute("select key from face where id=?", (request_string,))
    filename = ([i[0] for i in sel])
    delete = cursor.execute("DELETE FROM face WHERE id=?", (request_string,))
    conn.commit()
    conn.close()
    os.remove('images\\' + filename[0] + '.png')
    return '<OK>'

def update_record(request_string):
    request_string = request_string.split('<id>')
    keywords, date = parseDataFromPost(request_string[0])
    id_in_db = request_string[1]
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    upd = cursor.execute("UPDATE face SET rawstring = ?, keywords = ?, date = ? WHERE id = ?",
                         (request_string[0], keywords, date, id_in_db))
    conn.commit()
    conn.close()
    return "<OK>"
    
    
class MyHandler(BaseHTTPRequestHandler):
    store_path = pjoin(curdir, '1.jpg')
    
    def _set_headers(self):
        
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()

    def do_GET(self):
        
        self._set_headers()
        countRows = countOfRowsInDB()
        responseString = str(countRows)
        self.wfile.write(responseString.encode())
        

    def do_HEAD(self):
        self._set_headers()
        
    def do_POST(self):
        
        # Doesn't do anything with posted data
        self._set_headers()
        params = self.rfile.read(int(self.headers['Content-Length']))
        paramsSplit = params.split(b'<image>')
        method = paramsSplit[0].split(b'<method>')[0].decode()
        print(method)
        
        
        if method == 'appendToDb':
            dataString = paramsSplit[0].split(b'<method>')[1].decode()
            print(dataString)
            keywordsAndDate = parseDataFromPost(dataString)
            img = Image.open(io.BytesIO(paramsSplit[1]))
            image = cv2.cvtColor(numpy.array(img), cv2.COLOR_BGR2RGB)
            desc = getDescriptor(image)
            appendToDb(keywordsAndDate, dataString, desc, image)
            self.wfile.write("Succsess POST".encode())
            
        elif method == 'searchInDb':
            imbyte = params.split(b'<method>')[1]
            img = Image.open(io.BytesIO(imbyte))
            image = cv2.cvtColor(numpy.array(img), cv2.COLOR_BGR2RGB)
            desc = getDescriptor(image)
            searched = searchInDb(desc)
            print(searched)
            sendingString = "Succsess POST<content>" + searched
            self.wfile.write(bytes(str(len(sendingString)) + "<length>" + sendingString, 'utf-8').decode('unicode-escape').encode('ISO-8859-1'))        

        
        elif method == 'fullTextSearch':
            print(str(params.decode()))
            searchString = str(params.decode()).split('<text>')[1]
            file_bytes, byte_result = prepare_searched_response(searchString)
            
            if len(file_bytes) != 0 and len(byte_result) != 0:
                self.wfile.write(file_bytes + b'<textBegin>' + byte_result)
            
            else:
                self.wfile.write(b'<NOTFOUND>')
                
        elif method == 'delete_record':
            print(str(params.decode()))
            returned_string = delete_record(str(params.decode()).split("<text>")[1])
            self.wfile.write(returned_string.encode())
            
        elif method == 'update_record':
            print(str(params.decode()))
            returned_string = update_record(str(params.decode()).split("<text>")[1])
            self.wfile.write(returned_string.encode())
            
if __name__ == '__main__':
    server_class = HTTPServer
    httpd = server_class((HOST_NAME, PORT_NUMBER), MyHandler)
    print(time.asctime(), "Server Starts - %s:%s" % (HOST_NAME, PORT_NUMBER))
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        pass
    httpd.server_close()
    print(time.asctime(), "Server Stops - %s:%s" % (HOST_NAME, PORT_NUMBER))

Wed May 23 12:35:57 2018 Server Starts - localhost:1111


127.0.0.1 - - [23/May/2018 12:36:13] "POST / HTTP/1.1" 200 -


searchInDb



127.0.0.1 - - [23/May/2018 12:36:17] "POST / HTTP/1.1" 200 -


searchInDb



...

searchInDb
0.5123397557919628
0.715889737501809> 0.6
0.3820367556936943
<<row>>=<<row>>Дата регистрации=23.05.2018 14:30:15


127.0.0.1 - - [23/May/2018 14:31:53] "POST / HTTP/1.1" 200 -


searchInDb
0.6564353758777635> 0.6
0.6469822937476943> 0.6
0.510966460693129
<<row>>=<<row>>Дата регистрации=23.05.2018 14:30:15


127.0.0.1 - - [23/May/2018 14:31:59] "POST / HTTP/1.1" 200 -


searchInDb
0.6727999066035583> 0.6
0.6429181857341404> 0.6
0.5157763195080627
<<row>>=<<row>>Дата регистрации=23.05.2018 14:30:15
